# Recurrent coding strategies

Imagine we have this data:


In [1]:
IRdisplay::display_html('<iframe width="700" height="300" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pubhtml?gid=0&single=true"></iframe>')

And you need to create a cleaner version:

In [2]:
IRdisplay::display_html('<iframe width="700" height="300" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pubhtml?gid=2024244899&single=true"></iframe>')

In general, cleaning requires exploration. In exploration you check for inconsistencies.

The actual cleaning plan starts after exploration, and the  strategies to recover the true value of the cell can be one or several of these:

* **replace** the wrong value for good value.
* **detele** the wrong value.
* **extract** the good value.
* **split** the cell contents


**It is also recommended to clean the columns before the contents.**

Notice the **column names** have lightblue background,  and the **contents** have a grey background. I have also colored in red the column names that may serve as key columns, the unique identifiers.


## 1. On REGEX

REGEX is **complex**, but let share some _patterns_ that we will often use.

* This [a-zA-Z] represents any character of the alphabet (based in latin alphabet).
* This [^a-zA-Z] represents any characters *outside** the alphabet. The [^] means "the opposite" here. You can use for other purposes (see below).
* This \w is not the same as [a-zA-Z], \w can be equal to [0-9a-zA-Z\s_], that is, it includes numbers, spaces (\s) and underscore (_).
* Then, \W is the opposite of \w
* Similarly, \d is equal to [0-9], and \D is the opposite.
* Some other relevant symbols are:
  - The dot (**.**), representing any character.
  - The plus (**+**), here \d+ represents one or more digits.
  - The asterisk (__*__) here [a-z]* represents zero or more lowercase letters.
  - The symbols **^** and **\$** are also very relevant. Together the represent a whole string, that is,  ^\d[a-z]$ means the string starts with a digit and ends with a lowercase letter.

  You will need these when exploring and implementing the cleaning.

## 2. The Data Types

Imagine you CAN NOT see all the data. In that case we can use some code.

In [3]:
# the link as CSV
linkToData="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pub?gid=0&single=true&output=csv"

Read the data:

In [4]:
dirty=read.csv(linkToData,check.names=F)

I recommend you do this first:

In [5]:
str(dirty)

'data.frame':	6 obs. of  6 variables:
 $ identification : chr  "Perú" "USA" "Canada" "Côte D'Ivoire" ...
 $ identification2: chr  "Peru, South America" "USA, North America" "Canada, North America" "Côte D'Ivoire, Africa" ...
 $ var1           : chr  "1500" "2500" "3500" "2500" ...
 $ var 2          : chr  "1'200" "1'300" "--" "" ...
 $ var@3          : chr  "500" "$1 500" "1.5" "_" ...
 $ category       : chr  "a" "A" "Ba" "Ba" ...


Identify which are textual, numerical, or categorical.

* Columns **identification1** and **identification2** are *textual*.
* The columns from **var1** to **var@3** are all *numerical*. But if the type is _object_ the column should have some non numerical characters.
* Column **category** is *categorical*. Keep in mind that categorical types will NEVER be recognised as such when read from a CSV. They will always be understood as text (_object_).

The **column names** are always *textual*.

## 2. EXPLORATION


### 2.1. **Exploring TEXT**

When data is textual, you need to explore the cells to verify all the characters are part of the **alphabet**.

PANDAS offers **.str.** function that can only be used with text, if you try to use it with a numerical value, you will get an **error**.




Let's see how to use

In [6]:
# show me the cells that have a character outside the alphabet
dirty$identification[grep("[^a-zA-Z]",dirty$identification)]

[1] "Perú"           "Côte D'Ivoire"  "Israel [note]"  "United Kingdom"

United Kingdom is not dirty. But the space is outside the alphabet. What about:

In [7]:
dirty$identification[grep("\\W",dirty$identification)]

[1] "Côte D'Ivoire"  "Israel [note]"  "United Kingdom"

or...

In [8]:
dirty$identification[grep("[^\\w\\s]",dirty$identification,perl=T)]

[1] "Perú"          "Côte D'Ivoire" "Israel [note]"

Per**ú** is there. In general **\w** does not accept accented letters; but if you use Python 3, Perú will be there.

Then the safe option is:

In [9]:
dirty$identification[grep("[^a-zA-Z\\s]",dirty$identification,perl = T)]

[1] "Perú"          "Côte D'Ivoire" "Israel [note]"

A similar exploration should be done in the **column names**:

In [10]:
# allowing numbers, not spaces
names(dirty)[grep("[^0-9a-zA-Z]",names(dirty),perl = T)]

[1] "var 2" "var@3"

And in the case of the column with **categorical data**:

In [11]:
dirty$category[grep("[^a-zA-Z]",dirty$category,perl = T)]

[1] "?"

### 2.2. **Exploring NUMBERS**

If numbers are recognised as so, there is no cleaning needed. But if not, it means it has been recognised as text, then we use the regex **\d** (and its variations):

In [12]:
dirty$var1[grep("\\D",dirty$var1,perl = T)]

[1] "Dk"

In [13]:
dirty$'var 2'[grep("\\D",dirty$'var 2',perl = T)]

[1] "1'200" "1'300" "--"    "250k"

In [14]:
### Why the error?
# dirty$var@3[grep("\\D",dirty$var@3,perl = T)]

Notice I need to use **""** to access the variables with dirty names (space between words, and the **@** special character). That is why you clean the column names first:

In [15]:
dirty$'var@3'[grep("\\D",dirty$'var@3',perl=T)]

[1] "$1 500" "1.5"    "_"      "-"

There are cells with good values, but other values can not be kept. Use **\D** with care, numbers are complex. So I prefer something like this:

In [16]:
dirty$'var@3'[grep("[^\\d+\\.*\\d*]", dirty$'var@3', perl=T,invert = F)]

[1] "$1 500" "_"      "-"

## 2. CLEANING

As mentioned, cleaning may mean:

a. Making bad characters disappear.

b. Keeping good characters stay.


Let's start with the _column names_:

In [17]:
names(dirty)[grep("[^0-9a-zA-Z]",names(dirty),perl = T)]

[1] "var 2" "var@3"

How can you say: if "a space" or a "weird character", disappear? (that is *replace* by "")

In [18]:
# option 1
gsub("\\W",'',names(dirty), perl=T )


[1] "identification"  "identification2" "var1"            "var2"           
[5] "var3"            "category"

In [19]:
# option 2
gsub("[^\\w]",'',names(dirty), perl=T )

[1] "identification"  "identification2" "var1"            "var2"           
[5] "var3"            "category"

In [20]:
# # option 3
gsub("[^0-9a-zA-Z]",'',names(dirty), perl=T )

[1] "identification"  "identification2" "var1"            "var2"           
[5] "var3"            "category"

Choose any and make the change:

In [21]:
names(dirty)=gsub("[^0-9a-zA-Z]",'',names(dirty), perl=T )
dirty

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,"Peru, South America",1500,1'200,500,a
USA,"USA, North America",2500,1'300,$1 500,A
Canada,"Canada, North America",3500,--,1.5,Ba
Côte D'Ivoire,"Côte D'Ivoire, Africa",2500,,_,Ba
Israel [note],"Israel [note], Asia",Dk,250k,-,?
United Kingdom,"United Kingdom, Europe",2550,310000,330,Ba


The column names were cleaned by **Making bad characters disappear** 🙂


Let's check the **identification** column:

In [22]:
dirty$identification[grep("[^a-zA-Z\\s]",dirty$identification,perl = T)]

[1] "Perú"          "Côte D'Ivoire" "Israel [note]"

Not all characters detected are invalid. The **only** problem here is the brackets. Then:

* Option 1: Whatever inside brackets (including the brackets) have to go!

In [23]:
gsub("\\[.*\\]",'',dirty$identification,perl = T)

[1] "Perú"           "USA"            "Canada"         "Côte D'Ivoire" 
[5] "Israel "        "United Kingdom"

* Option 2: Splitting

In [24]:
strsplit(dirty$identification,split = '[',fixed=T)

[[1]]
[1] "Perú"

[[2]]
[1] "USA"

[[3]]
[1] "Canada"

[[4]]
[1] "Côte D'Ivoire"

[[5]]
[1] "Israel " "note]"  

[[6]]
[1] "United Kingdom"

You got a list. BUT you need a data frame column. Then:

In [25]:
## saving result
resultSplitIn2=strsplit(dirty$identification,split = '[',fixed=T)
# as matrix
goodColumn=c()
for (elements in resultSplitIn2){
  goodColumn=c(goodColumn,elements[1])

}
goodColumn

[1] "Perú"           "USA"            "Canada"         "Côte D'Ivoire" 
[5] "Israel "        "United Kingdom"

When you are happy, make the change:

In [26]:
dirty$identification=goodColumn
dirty

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,"Peru, South America",1500,1'200,500,a
USA,"USA, North America",2500,1'300,$1 500,A
Canada,"Canada, North America",3500,--,1.5,Ba
Côte D'Ivoire,"Côte D'Ivoire, Africa",2500,,_,Ba
Israel,"Israel [note], Asia",Dk,250k,-,?
United Kingdom,"United Kingdom, Europe",2550,310000,330,Ba


The **splitting** option seems very convenient for **identification2**:

In [27]:
## you want to keep [2]:
## saving result
resultSplitIn2=strsplit(dirty$identification2,split = ',', fixed = T)
# as matrix
goodColumn=c()
for (elements in resultSplitIn2){
  goodColumn=c(goodColumn,elements[2])

}
goodColumn

[1] " South America" " North America" " North America" " Africa"       
[5] " Asia"          " Europe"

If this is OK, then:

In [28]:
dirty$identification2=goodColumn
dirty

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,South America,1500,1'200,500,a
USA,North America,2500,1'300,$1 500,A
Canada,North America,3500,--,1.5,Ba
Côte D'Ivoire,Africa,2500,,_,Ba
Israel,Asia,Dk,250k,-,?
United Kingdom,Europe,2550,310000,330,Ba


The **category** requires a frequency table:

In [29]:
table(dirty$category)


 ?  A Ba  a 
 1  1  3  1 

You can conclude that the **a** is wrong, it should be **A**.

In [30]:
#what about:
gsub('a','A', dirty$category,fixed=T)

[1] "A"  "A"  "BA" "BA" "?"  "BA"

That changed **Ba** to **BA**!

In [31]:
## maybe
## ^: start of string
## $: end  of string
gsub('^a$','A', dirty$category)

[1] "A"  "A"  "Ba" "Ba" "?"  "Ba"

The simpler way:

In [32]:
dirty[dirty$category=='a','category']='A'

dirty

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,South America,1500,1'200,500,A
USA,North America,2500,1'300,$1 500,A
Canada,North America,3500,--,1.5,Ba
Côte D'Ivoire,Africa,2500,,_,Ba
Israel,Asia,Dk,250k,-,?
United Kingdom,Europe,2550,310000,330,Ba



As you seem there are some symbols for missing. We could change it now. Or later.

Let me first check the **numeric columns**:

In [33]:
gsub(',','',dirty$var1)


[1] "1500" "2500" "3500" "2500" "Dk"   "2550"

Then,

In [34]:
dirty$var1=gsub(',','',dirty$var1)
dirty


identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,South America,1500,1'200,500,A
USA,North America,2500,1'300,$1 500,A
Canada,North America,3500,--,1.5,Ba
Côte D'Ivoire,Africa,2500,,_,Ba
Israel,Asia,Dk,250k,-,?
United Kingdom,Europe,2550,310000,330,Ba


The **var2** is more complicated.

In [35]:
# save where you have the issue
dirty$var2_temp=grepl("\\'|k",dirty$var2,fixed=F)
dirty

identification,identification2,var1,var2,var3,category,var2_temp
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
Perú,South America,1500,1'200,500,A,TRUE
USA,North America,2500,1'300,$1 500,A,TRUE
Canada,North America,3500,--,1.5,Ba,FALSE
Côte D'Ivoire,Africa,2500,,_,Ba,FALSE
Israel,Asia,Dk,250k,-,?,TRUE
United Kingdom,Europe,2550,310000,330,Ba,FALSE


In [36]:
## now replace
dirty$var2=gsub("\\'|k",'',dirty$var2)
dirty

identification,identification2,var1,var2,var3,category,var2_temp
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
Perú,South America,1500,1200,500,A,TRUE
USA,North America,2500,1300,$1 500,A,TRUE
Canada,North America,3500,--,1.5,Ba,FALSE
Côte D'Ivoire,Africa,2500,,_,Ba,FALSE
Israel,Asia,Dk,250,-,?,TRUE
United Kingdom,Europe,2550,310000,330,Ba,FALSE


In [37]:
# now the real value
ifelse(dirty$var2_temp,paste0(dirty$var2,'000'),dirty$var2)

[1] "1200000" "1300000" "--"      ""        "250000"  "310000"

In [38]:
# then
dirty$var2=ifelse(dirty$var2_temp,paste0(dirty$var2,'000'),dirty$var2)
dirty$var2_temp=NULL
dirty

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,South America,1500,1200000,500,A
USA,North America,2500,1300000,$1 500,A
Canada,North America,3500,--,1.5,Ba
Côte D'Ivoire,Africa,2500,,_,Ba
Israel,Asia,Dk,250000,-,?
United Kingdom,Europe,2550,310000,330,Ba


The **var3** can be solved like this:

In [39]:
dirty['var3']=gsub("\\$|\\s",'',dirty$var3)
dirty

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,South America,1500,1200000,500,A
USA,North America,2500,1300000,1500,A
Canada,North America,3500,--,1.5,Ba
Côte D'Ivoire,Africa,2500,,_,Ba
Israel,Asia,Dk,250000,-,?
United Kingdom,Europe,2550,310000,330,Ba


### Detecting missing values:


Wrong missing values representation should be replace with care. Do it according to the data type.

Then, let's start with the **categorical** column:

In [40]:
badSymbolCat=grep('\\W+',dirty$category,value = T)
badSymbolCat

[1] "?"

Once found:

In [41]:
dirty$category=gsub(badSymbolCat,NA,dirty$category,fixed = T)
dirty

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,South America,1500,1200000,500,A
USA,North America,2500,1300000,1500,A
Canada,North America,3500,--,1.5,Ba
Côte D'Ivoire,Africa,2500,,_,Ba
Israel,Asia,Dk,250000,-,NA
United Kingdom,Europe,2550,310000,330,Ba


Let's go for the **numerical** cases:

In [42]:
dirty$var1[grep("[^\\d+\\.*\\d*]",dirty$var1,perl = T)]

[1] "Dk"

In [43]:
dirty$var2[grep("[^\\d+\\.*\\d*]",dirty$var2,perl = T)]

[1] "--"

In [44]:
dirty$var3[grep("[^\\d+\\.*\\d*]", dirty$var3, perl=T,invert = F)]

[1] "_" "-"

In [45]:
sapply(dirty[, c('var1','var2','var3')], function(col){col[grep("[^\\d+\\.*\\d*]", col, perl=T,invert = F)]})

$var1
[1] "Dk"

$var2
[1] "--"

$var3
[1] "_" "-"

In [46]:
unlist(sapply(dirty[, c('var1','var2','var3')], function(col){col[grep("[^\\d+\\.*\\d*]", col, perl=T,invert = F)]}))

var1  var2 var31 var32 
 "Dk"  "--"   "_"   "-"

In [47]:
unique(unlist(sapply(dirty[, c('var1','var2','var3')], function(col){col[grep("[^\\d+\\.*\\d*]", col, perl=T,invert = F)]})))

[1] "Dk" "--" "_"  "-"

Let's improve readability:

In [48]:
detectWrongNA= function(col){col[grep("[^\\d+\\.*\\d*]", col, perl=T,invert = F)]}
badSymbolNum=sapply(dirty[, c('var1','var2','var3')],detectWrongNA)
badSymbolNum_unlist=unlist(badSymbolNum)
badSymbolNum_vector=unique(badSymbolNum_unlist)
badSymbolNum_vector

[1] "Dk" "--" "_"  "-"

Let's clean those columns:

In [49]:


dirty[, c('var1','var2','var3')]=lapply(dirty[, c('var1','var2','var3')],function(col) ifelse((col %in% badSymbolNum_vector), NA, col))

dirty


identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,South America,1500,1200000,500,A
USA,North America,2500,1300000,1500,A
Canada,North America,3500,NA,1.5,Ba
Côte D'Ivoire,Africa,2500,,NA,Ba
Israel,Asia,NA,250000,NA,NA
United Kingdom,Europe,2550,310000,330,Ba


In [50]:
str(dirty)

'data.frame':	6 obs. of  6 variables:
 $ identification : chr  "Perú" "USA" "Canada" "Côte D'Ivoire" ...
 $ identification2: chr  " South America" " North America" " North America" " Africa" ...
 $ var1           : chr  "1500" "2500" "3500" "2500" ...
 $ var2           : chr  "1200000" "1300000" NA "" ...
 $ var3           : chr  "500" "1500" "1.5" NA ...
 $ category       : chr  "A" "A" "Ba" "Ba" ...


Always be preventive with meading and trailing spaces!

In [51]:
dirty[,]=sapply(dirty[,],trimws)
nowClean=dirty[,]
str(nowClean)

'data.frame':	6 obs. of  6 variables:
 $ identification : chr  "Perú" "USA" "Canada" "Côte D'Ivoire" ...
 $ identification2: chr  "South America" "North America" "North America" "Africa" ...
 $ var1           : chr  "1500" "2500" "3500" "2500" ...
 $ var2           : chr  "1200000" "1300000" NA "" ...
 $ var3           : chr  "500" "1500" "1.5" NA ...
 $ category       : chr  "A" "A" "Ba" "Ba" ...


In [52]:
nowClean

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Perú,South America,1500,1200000,500,A
USA,North America,2500,1300000,1500,A
Canada,North America,3500,NA,1.5,Ba
Côte D'Ivoire,Africa,2500,,NA,Ba
Israel,Asia,NA,250000,NA,NA
United Kingdom,Europe,2550,310000,330,Ba


## SAVING the CLEAN data

In [53]:
folder <- "DataCleanAndFormatted"

# Check if the folder exists
if (!dir.exists(folder)) {
  # Create the folder
  dir.create(folder)
  write.csv(nowClean,file.path(folder,"nowClean.csv"))

} else {
  write.csv(nowClean,file.path(folder,"nowClean.csv"))}